# COVID-19 QC

Dans ce document, nous voulons utiliser le modèle [SIR](https://code-for-philly.gitbook.io/chime/what-is-chime/sir-modeling) développé par *Penn Medecine* et l'appliquer à la réalité du Québec.

In [2]:
import os
import altair as alt
import streamlit as st
from datetime import date
import pandas as pd

#os.chdir("../../..")
from src.penn_chime.charts import (
    build_admits_chart,
    build_census_chart,
    build_census_chart_with_real,
    build_descriptions,
    build_sim_sir_w_date_chart,
    build_table,
)
from src.penn_chime.constants import DATE_FORMAT
from src.penn_chime.parameters import Parameters
from src.penn_chime.parameters import Parameters, Regions, Disposition

from src.penn_chime.models import SimSirModel
from src.penn_chime.settings import get_defaults
from src.penn_chime.presentation import (
    display_download_link,
    display_header,
    display_sidebar,
    hide_menu_style,
    write_definitions,
    write_footer,
)

Dans la prochaine cellule, on peut voir certains des paramètres du modèle SIR. Les paramètres QC_POPULATION (nombre d'habitants : 8,500,000), QC_MARKET_SHARE (couverture du système de santé : on considère l'ensemble des hopitaux, donc 100% = 1) et QC_DATE_PREMIER_HOSP (la date du premier cas hospitalisé : 15 mars 2020) sont connus et ne demandent pas d'analyse plus poussée.

Par contre, d'autres paramètres peuvent être estimés, déduits ou réfléchis à partir d'avis d'expert et des données du Québec :

- `QC_NB_JOURS_HOSPITALISATION`: nombre de jours en moyenne d'hospitalisation
- `QC_NB_JOURS_SI`: nombre de jours passé en moyenne aux soins intensifs
- `QC_POURC_HOSPITALISATION`: pourcentage des cas infectés (total et non rapportés) qui sont hospitalisés. Pour le moment nous l'avons estimé à partir du % moyen observé au Québec. Voir le fichier (`../../data_qc_ratios.csv`). Puisque les ratios sont calculés à partir du nombre de cas confirmés, et non du nombre de personnes infectées (qui est inconnu), nous l'avons ajusté à la baisse. Il est encore légèrement plus haut que le 5% rencontré dans la littérature sur le sujet.
- `QC_POURC_SI`: pourcentage des cas infectés (total et non rapportés) qui iront aux soins intensifs. Pour le moment nous l'avons estimé à partir du % moyen observé au Québec. Voir le fichier (`../../data_qc_ratios.csv`).
- `QC_DISTANT_SOCIAL`: facteur de ralentissement de la propagation du virus en réponse aux mesures mises en place et appliquées par la population (0%: aucune mesures appliquées et les contacts dans la population sont similaires à l'habitude) 

In [123]:
# Parametres propres au Quebec

QC_POPULATION = 8500000
QC_MARKET_SHARE = 1 # Couverture du systeme de santé

QC_DATE_PREMIER_HOSP = date(2020,3,15)

QC_NB_JOURS_HOSPITALISATION = 10
QC_NB_JOURS_SI = 12

QC_POURC_HOSPITALISATION = 0.06
QC_POURC_SI = 0.015
QC_POURC_VENTILATEUR = QC_POURC_SI

## Projections

### Projections à partir d'une date antérieure

Le graphique ci-dessous montre les cas hospitalisés et aux SI réels versus les prédictions du modèle en considérant la situation au Québec le 25 mars 2018 où il y avait 106 personnes hospitalisés.

In [124]:
# Change the default settings

QC_HOSP_CURRENT_DATE = 106
CURRENT_DATE = date(2020,3,25)
QC_DISTANTIATION_SOCIAL = 0

params = Parameters(
        population=QC_POPULATION,
        current_date=CURRENT_DATE,
        current_hospitalized=QC_HOSP_CURRENT_DATE,
        date_first_hospitalized=QC_DATE_PREMIER_HOSP,
        doubling_time=4.0,
        hospitalized=Disposition(QC_POURC_HOSPITALISATION, QC_NB_JOURS_HOSPITALISATION),
        icu=Disposition(QC_POURC_SI, QC_NB_JOURS_SI),
        infectious_days=14,
        market_share=QC_MARKET_SHARE,
        n_days=200,
        mitigation_date=CURRENT_DATE,
        relative_contact_rate=QC_DISTANTIATION_SOCIAL,
        ventilated=Disposition(QC_POURC_VENTILATEUR, QC_NB_JOURS_SI),
    )

# Run the Penn Med SIR model
p = display_sidebar(st, params)
m = SimSirModel(p)

Using doubling_time: 4.0
Set i_day = 26
Estimated date_first_hospitalized: 2020-02-28; current_date: 2020-03-25; i_day: 26
len(np.arange(-i_day, n_days+1)): 227
len(raw_df): 227


In [125]:
import datetime
DATES = ['Date']
COLS_MERGE = ['Date', 'hosp_reel', 'icu_reel']
qc_reels = pd.read_csv('analyses/data/qc_hosp_icu.csv', parse_dates=DATES)
qc_reels.rename(columns={'hosp':'hosp_reel',
                         'icu':'icu_reel'},
                 inplace=True)

df_with_real = m.census_floor_df.merge(qc_reels[COLS_MERGE], left_on='date', right_on='Date', how="left").drop(columns=['Date'])
df_with_real = df_with_real.astype({'hosp_reel':'float64', 'icu_reel': 'float64'})

In [126]:
# Graphique réel

data_real_plot = df_with_real.loc[df_with_real['date'] <= datetime.datetime(2020,4,5)]
data_real_plot
census_chart_with_real = build_census_chart_with_real(alt=alt, census_floor_df=data_real_plot, max_y_axis=p.max_y_axis)
census_chart_with_real

alt.LayerChart(...)

Notez que certains paramètres ont été manuellement ajustés (seulement légèrement) de sorte que les prédictions soient bonnes.

### Projections en date d'aujourd'hui

Les graphiques ci-dessous montrent les projections du modèle SIR, autant pour les nouveaux cas admis à tous les jours que pour le nombre de cas au cumulatif, selon les données receuillies en date du *XX* avril 2020.

In [133]:
# Change the default settings

QC_HOSP_CURRENT_DATE = 478
CURRENT_DATE = date(2020,4,3)
QC_DISTANTIATION_SOCIAL = 0.3

params = Parameters(
        population=QC_POPULATION,
        current_date=CURRENT_DATE,
        current_hospitalized=QC_HOSP_CURRENT_DATE,
        date_first_hospitalized=QC_DATE_PREMIER_HOSP,
        doubling_time=4.0,
        hospitalized=Disposition(QC_POURC_HOSPITALISATION, QC_NB_JOURS_HOSPITALISATION),
        icu=Disposition(QC_POURC_SI, QC_NB_JOURS_SI),
        infectious_days=14,
        market_share=QC_MARKET_SHARE,
        n_days=200,
        mitigation_date=CURRENT_DATE,
        relative_contact_rate=QC_DISTANTIATION_SOCIAL,
        ventilated=Disposition(QC_POURC_VENTILATEUR, QC_NB_JOURS_SI),
    )

# Run the Penn Med SIR model
p = display_sidebar(st, params)
m = SimSirModel(p)

Using doubling_time: 4.0
Set i_day = 35
Estimated date_first_hospitalized: 2020-02-28; current_date: 2020-04-03; i_day: 35
len(np.arange(-i_day, n_days+1)): 236
len(raw_df): 236


In [134]:
# Graphique nouveaux cas

admits_chart = build_admits_chart(alt=alt, admits_floor_df=m.admits_floor_df, max_y_axis=p.max_y_axis)
admits_chart

alt.LayerChart(...)

In [135]:
# Graphique cumulatif

census_chart = build_census_chart(alt=alt, census_floor_df=m.census_floor_df, max_y_axis=p.max_y_axis)
census_chart

alt.LayerChart(...)